In [4]:
import torch
import numpy as np
import datasets
import os
import umap
import evaluate
import accelerate
from mteb import MTEB
from pathlib import Path
from itertools import product
from IPython.core.debugger import set_trace
from datasets import Dataset, DatasetDict
from torch import nn
from torch.nn import functional as F
from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
from IPython.core.debugger import Pdb
from matplotlib import pyplot as plt
from transformers import AutoModel, AutoTokenizer
from pprint import pprint
from scipy.stats import spearmanr
from tqdm import tqdm
from multiprocessing import Pool
from itertools import chain
from numpy.lib.stride_tricks import sliding_window_view

from nbtools.sent_encoders.hf_models import from_hf
from nbtools.utils import (
    files,
    tensor_utils,
    display,
)
os.chdir('/data/john/projects/mltoolkit/')
cache_dir='./data/cache'
display.note(f'changed directory to \'{os.getcwd()}\'')

datasets.disable_caching()

# Set this to whatever you want
seed = 10

torch.manual_seed(seed)
np.random.seed(seed)

%load_ext autoreload
%autoreload 2
%matplotlib inline

[NOTE] changed directory to '/data/john/projects/mltoolkit'
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Encoder

In [3]:
model_name = 'mixedbread-ai/mxbai-embed-large-v1'
encoder = from_hf(
    model_name, 
    emb_dim=1024, 
    max_seq_len=512,
    cache_dir='./',
)

print(encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [5]:
TASK_LIST_CLASSIFICATION = [
    "AmazonCounterfactualClassification",
    "AmazonPolarityClassification",
    "AmazonReviewsClassification",
    "Banking77Classification",
    "EmotionClassification",
    "ImdbClassification",
    "MassiveIntentClassification",
    "MassiveScenarioClassification",
    "MTOPDomainClassification",
    "MTOPIntentClassification",
    "ToxicConversationsClassification",
    "TweetSentimentExtractionClassification",
]

TASK_LIST_CLUSTERING = [
    "ArxivClusteringP2P",
    "ArxivClusteringS2S",
    "BiorxivClusteringP2P",
    "BiorxivClusteringS2S",
    "MedrxivClusteringP2P",
    "MedrxivClusteringS2S",
    "RedditClustering",
    "RedditClusteringP2P",
    "StackExchangeClustering",
    "StackExchangeClusteringP2P",
    "TwentyNewsgroupsClustering",
]

TASK_LIST_PAIR_CLASSIFICATION = [
    "SprintDuplicateQuestions",
    "TwitterSemEval2015",
    "TwitterURLCorpus",
]

TASK_LIST_RERANKING = [
    "AskUbuntuDupQuestions",
    "MindSmallReranking",
    "SciDocsRR",
    "StackOverflowDupQuestions",
]

TASK_LIST_RETRIEVAL = [
    "ArguAna",
    "ClimateFEVER",
    "CQADupstackAndroidRetrieval",
    "CQADupstackEnglishRetrieval",
    "CQADupstackGamingRetrieval",
    "CQADupstackGisRetrieval",
    "CQADupstackMathematicaRetrieval",
    "CQADupstackPhysicsRetrieval",
    "CQADupstackProgrammersRetrieval",
    "CQADupstackStatsRetrieval",
    "CQADupstackTexRetrieval",
    "CQADupstackUnixRetrieval",
    "CQADupstackWebmastersRetrieval",
    "CQADupstackWordpressRetrieval",
    "DBPedia",
    "FEVER",
    "FiQA2018",
    "HotpotQA",
    "MSMARCO",
    "NFCorpus",
    "NQ",
    "QuoraRetrieval",
    "SCIDOCS",
    "SciFact",
    "Touche2020",
    "TRECCOVID",
]

TASK_LIST_STS = [
    "BIOSSES",
    "SICK-R",
    "STS12",
    "STS13",
    "STS14",
    "STS15",
    "STS16",
    "STS17",
    "STS22",
    "STSBenchmark",
    "SummEval",
]

TASK_LIST = (
    TASK_LIST_CLASSIFICATION
    + TASK_LIST_CLUSTERING
    + TASK_LIST_PAIR_CLASSIFICATION
    + TASK_LIST_RERANKING
    + TASK_LIST_RETRIEVAL
    + TASK_LIST_STS
)

In [6]:
for task in TASK_LIST:
    print(f'evaluating {model_name} on {task} ...')
    eval_splits = ["dev"] if task == "MSMARCO" else ["test"]
    evaluation = MTEB(
        tasks=[task], task_langs=["en"]
    )  # Remove "en" for running all languages
    evaluation.run(
        encoder, 
        output_folder=f'{files.project_root()}/results/mteb/{model_name}',
        eval_splits=eval_splits,
    )

evaluating mixedbread-ai/mxbai-embed-large-v1 on AmazonCounterfactualClassification ...


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AmazonCounterfactualClassification, s2s, multilingual 1 / 4 langs

/data/john/projects/nb/env/lib/python3.11/site-packages/datasets/load.py:1454: FutureWarning: The repository for mteb/amazon_counterfactual contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mteb/amazon_counterfactual
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/data/john/projects/nb/env/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` befo

...

evaluating mixedbread-ai/mxbai-embed-large-v1 on ClimateFEVER ...


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- ClimateFEVER, beir, s2p

/home/john/.cache/huggingface/datasets/BeIR/climate-fever.zip: 100%|██████████| 1.14G/1.14G [02:01<00:00, 10.1MiB/s]
Batches:  30%|██▉       | 116/391 [06:51<16:06,  3.52s/it]